In [1]:
% pylab inline
import os
import numpy as np
import pandas as pd

import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.5, allow_growth=True)

from keras.backend.tensorflow_backend import set_session
set_session(tf.Session(config = tf.ConfigProto(gpu_options = gpu_options)))

Populating the interactive namespace from numpy and matplotlib


/home/csh/.virtualenvs/tf-gpu/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from sys import path
import os
path.insert(0, "/home/csh/dev/evaxml/")

from preprocessing.encoders import SequenceIntEncoder

## Load sequences

In [3]:
def load_sequences(filename):
    from Bio import SeqIO
    proteins = {r.id: r.seq for r in SeqIO.parse(filename, 'fasta')}
    return proteins

## Load features

In [4]:
def load_features(filename):
    """ Load features into dataframes """

## Load partitions

In [5]:
def load_partitions(filename):
    import pandas as pd
    return pd.read_csv(filename, 
                       sep='\t', 
                       header=None, 
                       index_col=None, 
                       names=["ProteinID", "Partition", "Weight"])

## Batch encoder

In [6]:
def calc_epoch_size(weights):
    import numpy as np
    weights = weights / np.max( weights )
    return int(np.ceil(np.sum(weights)))

In [7]:
def generate_batch_indices(indices, weights, batch_size=64, seed=1, max_epoch_size=None):
    """
    indices          np.array (N)    List of data indices to sample from
    weights          np.array (N)    List of data weights to use for sampling probability
    batch_size       int             Number of proteins to sample from positive and negative
    
    returns (yield)  np.array (M)    Yields lists of indices (M=batch_size) indefinitely.
    """
    import numpy as np
    
    randgen = np.random.RandomState(seed)
    
    weights = weights / np.max( weights )
    probabilities = weights / np.sum( weights )
    epoch_size = calc_epoch_size(weights)
    assert batch_size < epoch_size
    
    while True:
        epoch_indices = randgen.choice(indices, size=epoch_size, replace=False, p=probabilities)
        for i in range(0, len(epoch_indices), batch_size):
            if len(epoch_indices[i:i + batch_size]) == batch_size:
                yield epoch_indices[i:i + batch_size]

In [8]:
def generate_batch(partitions_pos, partitions_neg, batch_size, target_values, sequences, blosum_matrix, seed=1, padding_length=None, max_epoch_size=None):
    from itertools import izip
    import random
    # Decide on maximum length
    if padding_length == None:
        padding_length = max( [len(s) for s in sequences.values()] )
    else:
        assert type(padding_length) == int
        
    for batch_pos, batch_neg in izip( generate_batch_indices(partitions_pos.ProteinID, partitions_pos.Weight, batch_size/2),
                                     generate_batch_indices(partitions_neg.ProteinID, partitions_neg.Weight, batch_size/2)):
        batch = list(batch_pos) + list(batch_neg)
        random.shuffle(batch) # inplace
        
        batch_sequences = sequences_int.loc[ batch ]#.as_matrix()
        batch_target_values = target_values.loc[batch]#.as_matrix()
        
        yield batch_sequences, batch_target_values

## Load and prepare data

In [9]:
partition_soluble = load_partitions("../data/partitioning/targettrack.soluble.partitions.csv")
partition_insoluble = load_partitions("../data/partitioning/targettrack.insoluble.partitions.csv")

In [10]:
sequences_soluble = load_sequences("../data/raw/targettrack.soluble.fasta")
sequences_insoluble = load_sequences("../data/raw/targettrack.insoluble.fasta")
sequences = dict(sequences_soluble, **sequences_insoluble)

In [11]:
sequences_int = SequenceIntEncoder(sequences, padding_length=1000, ignore="Z")

In [12]:
target_values = pd.concat([
    pd.Series(index=sequences_soluble.keys(), data=1),
    pd.Series(index=sequences_insoluble.keys(), data=0)
])

In [13]:
X, y = generate_batch(partitions_pos=partition_soluble, 
               partitions_neg=partition_insoluble, 
               batch_size=30, 
               target_values=target_values, 
               sequences=sequences, 
               blosum_matrix=None,
               padding_length=1000).next()

## Build model

In [14]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score
from keras.models import Sequential
from keras.models import Model
from keras.layers import Embedding
from keras.layers import Input
from keras.layers import merge
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Masking
from keras.layers import Merge
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D, MaxPooling2D, AveragePooling1D, AveragePooling2D
from keras.layers.pooling import GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers.wrappers import Bidirectional
from keras.optimizers import Adam, SGD, Adagrad, RMSprop
from keras.initializers import Ones, Zeros, Constant, RandomUniform, RandomNormal, glorot_normal, glorot_uniform
from keras import backend as K
from keras.layers import TimeDistributed
from keras.layers.core import RepeatVector
from keras.layers.core import Permute
from keras.layers.core import Lambda
from keras.layers.merge import Concatenate
from keras.layers.merge import Multiply
from keras.regularizers import l1, l2, l1_l2
from keras.constraints import max_norm, unit_norm, non_neg

#### Add custom metric

In [15]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """Wrapper for turning tensorflow metrics into keras metrics 
        
        Usage: 
        See https://stackoverflow.com/questions/45947351/how-to-use-tensorflow-metrics-in-keras
        """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

In [16]:
auc_roc = as_keras_metric(tf.metrics.auc)

In [17]:
mse =  as_keras_metric(tf.metrics.mean_squared_error)

In [18]:
@as_keras_metric
def auc_pr(y_true, y_pred, curve='PR'):
    return tf.metrics.auc(y_true, y_pred, curve=curve)

In [19]:
def compile_model(input_shape=(750, 20),
                          # convolutions
                          conv_filters=2, 
                          conv_filter_lengths=[2, 2], 
                          conv_strides=[1, 1],
                          conv_padding="same",
                          conv_pooling="average",
                          conv_pool_length=3,
                          conv_concat_axis=1,
                  
                          # recurrent
                          lstm=None,
                          lstm_bidirectional=True,
                          lstm_hidden=3,
                          lstm_drop_input=0,
                          lstm_drop_recurrent=0,
                          lstm_return_sequences=False,
                  
                          # dense
                          dense_hidden=4,
                          final_activation="sigmoid",
                          
                          # overall
                          bias_init="zeros",
                          dropout_probability=0.0,
                          loss="binary_crossentropy", # 'binary_crossentropy'
                          optimizer="rmsprop",
                          clipvalue=None,
                          clipnorm=None,
                          learning_rate=0.001,
                          decay=None,
                  
                          # metrics
                          metrics=['accuracy']
                         ):
    
    assert type(input_shape)==tuple,"Input shape must be a tuple"
    
    ######################################################################################################
    ######################################### Define multi-input #########################################
    ######################################################################################################
    # Sequence input
    l_input = Input(shape=input_shape, name='sequence-input')
    l_embed = Embedding(input_dim=21,
                        output_dim=20,
                        input_length=input_shape[0],
                        mask_zero=False,
                        )(l_input)

    ######################################################################################################
    ######################################### Convolutions ###########################################
    ######################################################################################################
    # sequence convolutions
    conv_layers = []
    for i in range(len(conv_filter_lengths)):
        c1_layer = Conv1D(filters=conv_filters, 
                         kernel_size=conv_filter_lengths[i], 
                         strides=conv_strides[i],
                         padding=conv_padding,
                         name='conv-seq-%d' % i)(l_embed)
        
        if conv_pooling == 'max':
            c1_layer = MaxPooling1D(pool_size=conv_pool_length, name='max-seq-%d' % i)(c1_layer)
        elif conv_pooling == 'average':
            c1_layer = AveragePooling1D(pool_size=conv_pool_length, name='av-seq-%d' % i)(c1_layer)
        elif conv_pooling == 'average_max':
            conv_pool_layers = []
            c1_layer1 = MaxPooling1D(pool_size=conv_pool_length)(c1_layer)
            conv_pool_layers.append(c1_layer1)
            c1_layer2 = AveragePooling1D(pool_size=conv_pool_length)(c1_layer)
            conv_pool_layers.append(c1_layer2)
            c1_layer = Concatenate(axis=conv_concat_axis, name='av-max-seq-%d' % i)(conv_pool_layers)
        conv_layers.append( c1_layer )
        
    # Merging all filters into one set
    l_main = Concatenate(axis=conv_concat_axis, name='merge-conv')(conv_layers)
    ######################################################################################################
    ######################################### Normalization ##############################################
    ######################################################################################################
    l_main = BatchNormalization(name="normalization")(l_main)
    
    ######################################################################################################
    ######################################### Dropouts ###################################################
    ######################################################################################################
    l_main = Dropout(dropout_probability)(l_main)
    
    ######################################################################################################
    ######################################### Recurrents #################################################
    ######################################################################################################
    if lstm:
        if not lstm_bidirectional:
            l_main = LSTM(lstm_hidden,
                              dropout=rnn_drop_input,
                              recurrent_dropout=lstm_drop_recurrent,
                              return_sequences=lstm_return_sequences, name='uni-LSTM')(l_main)
        else:
            l_main = Bidirectional(LSTM(lstm_hidden,
                                          dropout=lstm_drop_input,
                                          recurrent_dropout=lstm_drop_recurrent,
                                          return_sequences=lstm_return_sequences), name='bi-LSTM')(l_main)
    
    if lstm_return_sequences==True:
        l_main = Flatten()(l_main)
    
    ######################################################################################################
    ######################################### Dense learning #############################################
    ######################################################################################################
    l_main = Dense(dense_hidden)(l_main)
    
    ######################################################################################################
    ######################################### Dense outputs ##############################################
    ######################################################################################################
    l_output = Dense(1, activation=final_activation, name='output')(l_main)
    
    ######################################################################################################
    ######################################### Wrap model #################################################
    ######################################################################################################
    model = Model(outputs=[l_output], inputs=[l_input])
    
    # Prepare optimizer
    #optimizer_opts = dict(clipvalue=clipvalue)
    optimizer_opts = dict()
    if learning_rate:
        optimizer_opts.update(lr=learning_rate)
    if decay:
        optimizer_opts.update(decay=decay)
    if clipvalue:
        optimizer_opts.update(clipvalue=clipvalue)
    if clipnorm:
        optimizer_opts.update(clipnorm=clipnorm)
        
    if optimizer.lower()=='adam':
        optimizer = Adam(**optimizer_opts)
    elif optimizer.lower()=='adagrad':
        optimizer = Adagrad(**optimizer_opts)
    elif optimizer.lower()=='sgd':
        optimizer = SGD(**optimizer_opts)
    elif optimizer.lower()=='rmsprop':
        optimizer = RMSprop(**optimizer_opts)
    elif optimizer.lower()=='amsgrad':
        optimizer_opts.update(amsgrad=True)
        optimizer = Adam(**optimizer_opts)
    else:
        raise ValueError("Unknown optimizer '%s'. Valid options are 'adam', 'adagrad' and 'sgd'" % optimizer)
        
    # Compile model
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
    # Enable predict_proba method for AUC scoring
    model.predict_proba = model.predict    
    
    return model

In [20]:
def get_model():
    auc_roc = as_keras_metric(tf.metrics.auc)
    return compile_model(input_shape=(1000,),
                                      # convolutions
                                      conv_filters=3, 
                                      conv_filter_lengths=[2, 8, 16], 
                                      conv_strides=[2, 2, 2],
                                      conv_padding="same",
                                      conv_pooling="average",
                                      conv_pool_length=10,
                                      conv_concat_axis=2,

                                      # recurrent
                                      lstm=True,
                                      lstm_bidirectional=True,
                                      lstm_hidden=5,
                                      lstm_drop_input=0,
                                      lstm_drop_recurrent=0,
                                      lstm_return_sequences=True,

                                      # dense
                                      dense_hidden=16,
                                      final_activation="sigmoid",

                                      # overall
                                      bias_init="zeros",
                                      dropout_probability=0.0,
                                      loss="binary_crossentropy", # 'binary_crossentropy'
                                      optimizer="rmsprop",
                                      clipvalue=0.5,
                                      clipnorm=None,
                                      learning_rate=0.005,
                                      decay=None,
                               
                                      #metrics
                                      metrics=[auc_roc, mse])

In [21]:
model = get_model()
model.summary()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence-input (InputLayer)     (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 20)     420         sequence-input[0][0]             
__________________________________________________________________________________________________
conv-seq-0 (Conv1D)             (None, 500, 3)       123         embedding_1[0][0]                
__________________________________________________________________________________________________
conv-seq-1 (Conv1D)             (None, 500, 3)       483         embedding_1[0][0]                
__________________________

## Train

In [22]:
def cv_split(parts, n_test=1):
    """ Return combinations (train,test) of partition indices for cross validation """
    from itertools import combinations
    if type(parts) == int:
        parts = range(parts)
    for test in combinations( parts, n_test ):
        yield [x for x in parts if x not in test], list(test)

In [23]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau

In [ ]:
model_prefix = "testing_ckt"

batch_size = 512
selected_partitions = [0,1,2,3] # 5'th partition is completely left out.


model_id = 0
for train_partitions_indices, test_partition_indices in cv_split( selected_partitions, 1 ):
    if 3 in test_partition_indices:
        continue # Limit training of only 3 models
    
    model = get_model()
    
    print "Training model %d on training partitions %s and test partitions %s" % (model_id, str(train_partitions_indices), str(test_partition_indices))

    # Prepare training partitions
    partitions_pos_train = partition_soluble[ partition_soluble.Partition.isin( train_partitions_indices ) ]
    partitions_neg_train = partition_insoluble[ partition_insoluble.Partition.isin( train_partitions_indices ) ]

    # Prepare earlystoping data
    partitions_pos_test = partition_soluble[ partition_soluble.Partition.isin( test_partition_indices ) ]
    partitions_neg_test = partition_insoluble[ partition_insoluble.Partition.isin( test_partition_indices ) ]

    sequences_int_test = sequences_int.loc[list(partitions_pos_test.ProteinID) + list(partitions_neg_test.ProteinID)]
    target_values_test = target_values.loc[list(partitions_pos_test.ProteinID) + list(partitions_neg_test.ProteinID)]

    # Use positive data to estimate epoch size
    epoch_size = calc_epoch_size(partitions_pos_train.Weight)

    print "   Training data: %d" % (len(partitions_pos_train) + len(partitions_neg_train))
    print "   Testing data: %d" % len(sequences_int_test)
    
    

    early_stopping = EarlyStopping(monitor='val_auc',
                                   mode='max',
                                   patience=50, 
                                   min_delta=0.0001,
                                   verbose=1)

    model_checkpoint = ModelCheckpoint('models/%s_%d.h5' % (model_prefix, model_id),
                                       monitor='val_auc',
                                       mode='max',
                                       verbose=0,
                                       save_best_only=True,
                                       save_weights_only=True)

    reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_auc',
                                             mode='max',
                                             factor=0.5,
                                             patience=8,
                                             verbose=1,
                                             cooldown=10,
                                             min_lr=0.00001)

    tensorboard = TensorBoard(log_dir="./graph/%s_%d" % (model_prefix, model_id), 
                              write_images=True,
                              histogram_freq=0)

    validation_data = (sequences_int_test, target_values_test)

    model.fit_generator( generate_batch(partitions_pos_train, 
                                        partitions_neg_train,
                                        batch_size=batch_size,
                                        target_values=target_values,
                                        sequences=sequences,
                                        blosum_matrix=None,
                                        padding_length=1000),
                         steps_per_epoch = epoch_size/(batch_size/2),
                         epochs = 500,
                         callbacks=[early_stopping, model_checkpoint, reduce_lr_on_plateau, tensorboard],
                         validation_data=validation_data,
                         verbose = 1)

    model_id += 1

Training model 0 on training partitions [1, 2, 3] and test partitions [0]
   Training data: 29607
   Testing data: 17650
Epoch 1/500
42/42 [==============================] - 15s 350ms/step - loss: 0.9072 - auc: 0.4983 - mean_squared_error: 0.3216 - val_loss: 0.7713 - val_auc: 0.5478 - val_mean_squared_error: 0.2786
Epoch 2/500
42/42 [==============================] - 13s 309ms/step - loss: 0.6473 - auc: 0.5353 - mean_squared_error: 0.2809 - val_loss: 0.6385 - val_auc: 0.5817 - val_mean_squared_error: 0.2620
Epoch 3/500
42/42 [==============================] - 13s 310ms/step - loss: 0.6065 - auc: 0.5969 - mean_squared_error: 0.2553 - val_loss: 0.7996 - val_auc: 0.6335 - val_mean_squared_error: 0.2456
Epoch 4/500
42/42 [==============================] - 12s 295ms/step - loss: 0.5849 - auc: 0.6274 - mean_squared_error: 0.2506 - val_loss: 0.7849 - val_auc: 0.6490 - val_mean_squared_error: 0.2436
Epoch 5/500
42/42 [==============================] - 12s 286ms/step - loss: 0.5764 - auc: 0.644

42/42 [==============================] - 12s 283ms/step - loss: 0.5139 - auc: 0.7750 - mean_squared_error: 0.1895 - val_loss: 0.4984 - val_auc: 0.7754 - val_mean_squared_error: 0.1894
Epoch 43/500
42/42 [==============================] - 12s 291ms/step - loss: 0.5170 - auc: 0.7758 - mean_squared_error: 0.1892 - val_loss: 0.4906 - val_auc: 0.7756 - val_mean_squared_error: 0.1893
Epoch 44/500
42/42 [==============================] - 12s 293ms/step - loss: 0.5167 - auc: 0.7766 - mean_squared_error: 0.1888 - val_loss: 0.5307 - val_auc: 0.7771 - val_mean_squared_error: 0.1887
Epoch 45/500
42/42 [==============================] - 12s 284ms/step - loss: 0.5123 - auc: 0.7771 - mean_squared_error: 0.1887 - val_loss: 0.4994 - val_auc: 0.7774 - val_mean_squared_error: 0.1885
Epoch 46/500
42/42 [==============================] - 12s 296ms/step - loss: 0.5161 - auc: 0.7779 - mean_squared_error: 0.1884 - val_loss: 0.5355 - val_auc: 0.7783 - val_mean_squared_error: 0.1882
Epoch 47/500
42/42 [========

In [ ]:
from sklearn.metrics import roc_auc_score
y_pred = model.predict(validation_data[0], verbose=1)

print roc_auc_score(validation_data[1], y_pred.flatten())

## Look at history output

In [ ]:
history = histories[0]

In [ ]:
fig = plt.figure()
#['acc', 'loss', 'val_acc', 'val_loss']
x = range(len(history.history.values()[0]))
ax = fig.add_subplot(111)
line, = ax.plot(x, history.history["auc"], color='b', linewidth="1")
line, = ax.plot(x, history.history["val_auc"], color='r', linewidth="1")
ax.set_title("%d" % (i+1))
lgd = ax.legend(["AUC (train)", "AUC (stop)"], loc="upper left", bbox_to_anchor=(1,0.905))
fig.tight_layout()

In [ ]:
fig = plt.figure()
#['acc', 'loss', 'val_acc', 'val_loss']
x = range(len(history.history.values()[0]))
ax = fig.add_subplot(111)
line, = ax.plot(x, history.history["loss"], color='b', linewidth="1")
line, = ax.plot(x, history.history["val_loss"], color='r', linewidth="1")
ax.set_title("%d" % (i+1))
lgd = ax.legend(["Log-loss (train)", "Log-loss (stop)"], loc="upper left", bbox_to_anchor=(1,0.905))
fig.tight_layout()